<a href="https://colab.research.google.com/github/ShuqairABD/Gradio_interface_ResNet_152_-/blob/main/Abdalla_Shuqair/Gradio_interface_ResNet_152_%D0%9C%D0%9E%D0%99%D0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **start**

In [ ]:
!pip install gradio


In [ ]:
!pip install pydicom

In [ ]:
import gradio as gr
import torch
from torchvision import models, transforms
import torch.nn as nn
import os
import numpy as np
import pydicom
from PIL import Image

import os
import zipfile
import tempfile

## **Gradio in folder**

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ResNet-152 with weights! (можно с весами)!!
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)
#model = models.resnet152(weights=None)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 1),
    nn.Sigmoid()
).to(device)

model_path = '/content/drive/MyDrive/bouns_fine_tuned_resnet152.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

label_mapping = {0: 'Healthy', 1: 'UnHealthy'}

def classify_zip(zip_file):
    results = []
    with tempfile.TemporaryDirectory() as tmpdirname:
        # ZIP
        with zipfile.ZipFile(zip_file.name, 'r') as zip_ref:
            zip_ref.extractall(tmpdirname)

        for root, _, files in os.walk(tmpdirname):
            for filename in files:
                if filename.lower().endswith('.dcm'):
                    dicom_image = pydicom.dcmread(os.path.join(root, filename))
                    image_array = dicom_image.pixel_array

                    if image_array.max() > 1:
                        image_array = (image_array / image_array.max() * 255).astype(np.uint8)
                    image = Image.fromarray(image_array).convert('RGB')

                    input_image = transform(image).unsqueeze(0).to(device)


                    with torch.no_grad():
                        output = model(input_image)
                    predicted = (output > 0.5).float()
                    predicted_label = predicted.item()
                    predicted_class_name = label_mapping[int(predicted_label)]

                    # результат ------>> list
                    results.append((image, f"{filename}: {predicted_class_name}"))

    return results

# Gradio
iface = gr.Interface(
    fn=classify_zip,
    inputs=gr.components.File(label="Загрузить zip-файл DICOM"),
    outputs=gr.components.Gallery(label="Предсказанные классы"),
    title="Классификация патология / норма с помощью ResNet-152",
    description="Загрузить ZIP-файл, содержащий файлы DICOM, чтобы классифицировать их как норма или патология"
)

iface.launch()


Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:01<00:00, 132MB/s]
<ipython-input-4-4b29ffa35dc8>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1b82c762c1c0738810.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
